In [1]:
import json
import os
from collections import Counter, defaultdict, OrderedDict
import copy

In [2]:
test = defaultdict(Counter)
for file in os.listdir("data"):
    if "small" in file:
        continue
    if file.startswith("p-raw"):
        with open(fr"data/{file}") as f:
            data = json.load(f)
            for key in list(data.keys()):
                if key.startswith("metadata"):
                    continue
                else:
                    for term in data[key]['terms']:
                        # print(term)
                        # if term in test: #test[term] already exists, keep counting
                        #     continue
                        # else:
                        #     test[term] = 0 #init the search term to the test dict then keep counting
                        
                        # print(test[term])
                        try:
                            for tech in data[key]['techs'][0].split("\n"):
                                clean_tech = tech.strip()
                                clean_tech = clean_tech.replace("-", "")
                                if tech in test[term]:
                                    test[term][clean_tech] += 1
                                else:
                                    test[term][clean_tech] = 1
                        except IndexError:
                            print(f"There were no techs in {key} (term: {term})")

    else:
        continue

There were no techs in 5a40ec2b5cfe588e (term: data science)
There were no techs in 2b038a24f2df9b91 (term: data science)
There were no techs in 7808b987a7beb309 (term: data science)
There were no techs in 7808b987a7beb309 (term: machine learning engineer)
There were no techs in 7c9aa7118a17b91d (term: data science)
There were no techs in 89e03009fcbaa993 (term: data science)
There were no techs in 78eaff2340fd9e6f (term: data science)
There were no techs in 28502ee624b77b4a (term: data science)
There were no techs in 6a00657c1c0f440a (term: data science)
There were no techs in b0a28a235289a06f (term: data science)
There were no techs in 698f0759473b8f72 (term: data science)
There were no techs in 17182aa90f7849f0 (term: data science)
There were no techs in 6045eea523ab5375 (term: data science)
There were no techs in f870f79476915e99 (term: data science)
There were no techs in 3660972e017d75b2 (term: data analyst)
There were no techs in 9ef4525babacd3aa (term: data analyst)
There were 

In [3]:
OrderedDict(sorted(test["data science"].items(), key=lambda x: float(x[1]), reverse=True))

OrderedDict([('python', 167),
             ('sql', 83),
             ('tableau', 29),
             ('r', 26),
             ('uipath', 26),
             ('tensorflow', 21),
             ('none', 16),
             ('excel', 16),
             ('aws', 15),
             ('llms', 15),
             ('machine learning', 15),
             ('sas', 14),
             ('soc2', 14),
             ('power bi', 13),
             ('java', 12),
             ('pytorch', 10),
             ('collaborating across departments', 9),
             ('apache spark', 8),
             ('databricks', 6),
             ('hadoop', 6),
             ('nlp', 5),
             ('generative ai', 5),
             ('apis', 5),
             ('macros', 5),
             ('artificial intelligence', 5),
             ('ai', 5),
             ('jira', 5),
             ('iot', 5),
             ('automated tools', 5),
             ('pandas', 5),
             ('agile', 5),
             ('ml best practices', 5),
             ('statistics',

In [4]:
# Create ordered dict of counts by keyword search
ord = {k: OrderedDict(sorted(v.items(), key=lambda x: float(x[1]), reverse=True)) for k, v in test.items()}

In [5]:
list(ord['data science'].items())[:5]

[('python', 167), ('sql', 83), ('tableau', 29), ('r', 26), ('uipath', 26)]

In [6]:
for key in ord.keys():
    top5 = list(ord[key].items())[:5]
    print(f"Top 5 skills for {key}: {top5} \n")

Top 5 skills for data science: [('python', 167), ('sql', 83), ('tableau', 29), ('r', 26), ('uipath', 26)] 

Top 5 skills for data analyst: [('systems development lifecycle cycle', 68), ('sql', 60), ('azure purview', 53), ('tableau', 38), ('none', 33)] 

Top 5 skills for data engineer: [('python', 59), ('aws', 40), ('sql', 24), ('security clearance', 12), ('java', 11)] 

Top 5 skills for machine learning engineer: [('cad/cam software', 250), ('python', 128), ('controllers (plcs)', 31), ('tensorflow', 30), ('none', 29)] 

Top 5 skills for mlops: [('aws', 42), ('python', 22), ('terraform', 22), ('llms', 16), ('java', 12)] 



# Remove the things we don't want from our dictionary.  
Will still be in the json if we ever want to do something with them.  Right now these are 'none' and things which contain the words 'system development lifecycle'


In [19]:
ordcopy = copy.deepcopy(ord)

bad_words = ["none", "systems development lifecycle cycle", "security clearance", "devops engineer"]

for k1, v1 in ord.items():
    for k2, v2 in v1.items():
        if k2 in bad_words:
            print(f"k2: {k2}")
            del ordcopy[k1][k2]
        # if k2 == "none":
        #     print(f"k2: 'none'")
        #     del ordcopy[k1][k2]
        # elif "systems development lifecycle" in k2:
        #     print(f"k2: {k2}")
        #     del ordcopy[k1][k2]
        else:
            continue

k2: none
k2: systems development lifecycle cycle
k2: none
k2: security clearance
k2: none
k2: none
k2: devops engineer
k2: none


In [21]:
n = 10
for key in ordcopy.keys():
    topN = list(ordcopy[key].items())[:n]
    print(f"Top {n} skills for {key}: {topN} \n")

Top 10 skills for data science: [('python', 167), ('sql', 83), ('tableau', 29), ('r', 26), ('uipath', 26), ('tensorflow', 21), ('excel', 16), ('aws', 15), ('llms', 15), ('machine learning', 15)] 

Top 10 skills for data analyst: [('sql', 60), ('azure purview', 53), ('tableau', 38), ('excel', 30), ('alteryx', 19), ('jira', 15), ('power bi', 14), ('ms excel', 13), ('salesforce', 11), ('microsoft excel', 10)] 

Top 10 skills for data engineer: [('python', 59), ('aws', 40), ('sql', 24), ('java', 11), ('snowflake', 10), ('vmware vsphere', 10), ('databricks', 9), ('data modelling', 9), ('ansys software', 8), ('apache spark', 7)] 

Top 10 skills for machine learning engineer: [('cad/cam software', 250), ('python', 128), ('controllers (plcs)', 31), ('tensorflow', 30), ('machine learning', 24), ('sql', 18), ('llms', 16), ('java', 15), ('c++', 13), ('aws', 12)] 

Top 10 skills for mlops: [('aws', 42), ('python', 22), ('terraform', 22), ('llms', 16), ('java', 12), ('docker', 8), ('jenkins', 7), (